In [1]:
data_path = '../dataset'

In [2]:
import os

In [3]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

../dataset/test
../dataset/train
../dataset/val


In [4]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')
test_data_path = os.path.join(data_path, 'test')

In [5]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

{0: 'bart_simpson',
 1: 'charles_montgomery_burns',
 2: 'homer_simpson',
 3: 'krusty_the_clown',
 4: 'lisa_simpson',
 5: 'marge_simpson',
 6: 'milhouse_van_houten',
 7: 'moe_szyslak',
 8: 'ned_flanders',
 9: 'principal_skinner'}

In [6]:
val_classes = dict()

for path in sorted(os.listdir(val_data_path)):
    if os.path.isdir(os.path.join(val_data_path, path)):
        val_classes.setdefault(len(val_classes), path)
        
val_classes

{0: 'bart_simpson',
 1: 'charles_montgomery_burns',
 2: 'homer_simpson',
 3: 'krusty_the_clown',
 4: 'lisa_simpson',
 5: 'marge_simpson',
 6: 'milhouse_van_houten',
 7: 'moe_szyslak',
 8: 'ned_flanders',
 9: 'principal_skinner'}

In [7]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

{0: 'bart_simpson',
 1: 'charles_montgomery_burns',
 2: 'homer_simpson',
 3: 'krusty_the_clown',
 4: 'lisa_simpson',
 5: 'marge_simpson',
 6: 'milhouse_van_houten',
 7: 'moe_szyslak',
 8: 'ned_flanders',
 9: 'principal_skinner'}

In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [10]:
torch.cuda.empty_cache()

In [11]:
from torchvision import transforms as T

In [12]:
train_transform = T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
val_transform = T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
test_transform = T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
from torchvision.datasets import ImageFolder

In [16]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=train_transform
)

In [17]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 6500
    Root location: ../dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [18]:
val_dataset = ImageFolder(
    root=val_data_path,
    transform=val_transform
)

In [19]:
val_dataset

Dataset ImageFolder
    Number of datapoints: 1500
    Root location: ../dataset/val
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [20]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=test_transform
)

In [21]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2000
    Root location: ../dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [22]:
from torch.utils.data import DataLoader

In [23]:
BATCH_SIZE = 16

In [24]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [25]:
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [26]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [27]:
from torchvision import models

In [28]:
model = models.resnet18(pretrained=True)

In [29]:
for param in model.parameters():
    param.requires_grad = False

In [30]:
import torch.nn as nn

In [31]:
n_inputs = model.fc.in_features
n_outputs = 10

In [32]:
sequential_layers = nn.Sequential(
    nn.Linear(n_inputs, 128),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.Linear(128, n_outputs),
    nn.LogSoftmax(dim=1)
)

In [33]:
model.fc = sequential_layers

In [34]:
model = model.to('cuda')

In [35]:
import torch.optim as optim

In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=.1)

In [37]:
loaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [38]:
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}

In [39]:
%%time

EPOCHS = 15

for epoch in range(1, EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS}\n{'='*25}")
    for phase in ['train', 'val']:
        running_loss = .0
        running_corrects = .0
        if phase == 'train': model.train()
        if phase == 'val': model.eval()
        for inputs, labels in loaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            res = torch.sum(preds == labels)
            running_corrects += res
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        if phase == 'train': scheduler.step()
        print(f"Loss ({phase}): {epoch_loss}, Acc ({phase}): {epoch_acc}")


Epoch 1/15
Loss (train): 2.1852115738208475, Acc (train): 0.2056923076923077
Loss (val): 1.9974153626759847, Acc (val): 0.30466666666666664

Epoch 2/15
Loss (train): 1.9232732991438646, Acc (train): 0.34184615384615386
Loss (val): 1.7794704548517863, Acc (val): 0.412

Epoch 3/15
Loss (train): 1.7621381853543796, Acc (train): 0.39984615384615385
Loss (val): 1.7083666667938233, Acc (val): 0.44066666666666665

Epoch 4/15
Loss (train): 1.7034465722304124, Acc (train): 0.4290769230769231
Loss (val): 1.6857879638671875, Acc (val): 0.4473333333333333

Epoch 5/15
Loss (train): 1.6660019892912645, Acc (train): 0.4293846153846154
Loss (val): 1.6772985127766926, Acc (val): 0.42933333333333334

Epoch 6/15
Loss (train): 1.603434271959158, Acc (train): 0.4612307692307692
Loss (val): 1.6624434502919514, Acc (val): 0.44866666666666666

Epoch 7/15
Loss (train): 1.5827630645311797, Acc (train): 0.47000000000000003
Loss (val): 1.646861976623535, Acc (val): 0.4593333333333333

Epoch 8/15


KeyboardInterrupt: 

In [40]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
%%time

for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels)
    
loss = running_loss / dataset_sizes['test']
acc = running_corrects.double() / dataset_sizes['test']

In [ ]:
print(f"Test Loss: {epoch_loss}, Test Accuracy: {epoch_acc}")